# Transient Heat Condiction Problem

$$ \frac{\partial \rho T}{\partial t} = \frac{\partial}{\partial x} (\frac{k}{c_p} \frac{\partial T}{\partial x}) $$

Considering the Finite Volume Method (FVM), we'll solve the unidimensional equation for T by integrating both sides over a control volume (from a face in position $x$ to a face in the position $x+dx$) and over time (from time $t$ to $t+dt$), as in

$$ \int_{x}^{x+\Delta x} \int_{t}^{t+\Delta t} \frac{\partial \rho T}{\partial t} dt\ dx = \int_{x}^{x+\Delta x} \int_{t}^{t+\Delta t} \frac{\partial}{\partial x} (\frac{k}{c_p} \frac{\partial T}{\partial x}) dt\ dx$$

Imagining an 1D domain, represented using several control volume elements (As shown below), we then have that $x$ = $T_w$, and $x+\Delta x$ = $T_e$ (That is, the property available at the domain's limits, not their centers).

<img src="domain_example.png">

This results in

$$ \int_{w}^{e} (\rho T_P - \rho T_P^o) dx = \int_{t}^{t+\Delta t} \frac{k}{c_p} \frac{\partial T}{\partial x}\bigg\rvert_e - \frac{k}{c_p} \frac{\partial T}{\partial x}\bigg\rvert_w dt$$

Let now assume that tha property is constant over $T_X$ from face $Tw$ to face $Te$. Let's also not specify the behavior of the integral of the properties on the right side of the equation, letting $\theta$ be a variable that will specify it later (As done in [1]), then we have

$$ (\rho T_P - \rho T_P^o)\Delta x = (\frac{k}{c_p} \frac{\partial T}{\partial x}\bigg\rvert_e^\theta - \frac{k}{c_p} \frac{\partial T}{\partial x}\bigg\rvert_w^\theta)\Delta t$$

To approximate the values of the derivative of $T$ over the faces $Tw$ and $Te$, it'll be used a common central differences approach, that is

$$\frac{\Delta F}{\Delta x} = \frac{F(x + \Delta x) - F(x - \Delta x)}{\Delta x}$$

Remember, again, that $F(x-\Delta x) = F_w$ and $F(x+\Delta x) = F_e$, then

$$ (\rho T_P - \rho T_P^o)\Delta x = (\frac{k}{c_p} \frac{T_E^\theta - T_P^\theta}{\Delta x} - \frac{k}{c_p} \frac{T_P^\theta - T_W^\theta}{\Delta x})\Delta t$$

Where $T^\theta = \theta T + (1 - \theta) T^o$

## Explicit Formulation

For the explicit case, $\theta = 0$, and the equation will be simply $T_P = f(T_P^o, T_W^o, T_E^o)$, as shown bellow:

$$ (\rho T_P - \rho T_P^o)\Delta x = (\frac{k}{c_p} \frac{T_E^o - T_P^o}{\Delta x} - \frac{k}{c_p} \frac{T_P^o - T_W^o}{\Delta x})\Delta t$$

So, $T_P$ is the only unknown variable at this point. For the case of the first timestep, $T_P^o$, $T_E^o$ and $T_W^o$ must be known. Also, for the boundaries, values for $T_E$ and $T_W$ must be specified. This leads to the following equation:

$$\frac{\rho \Delta x}{\Delta t} T_P = \frac{k}{c_p \Delta x} T_E^o - 2\frac{k}{c_p \Delta x} T_P^o + \frac{k}{c_p \Delta x} T_W^o + \frac{\rho \Delta x}{\Delta t} T_P^o$$

# Bibliografy

[1] MALISKA, C. R. - Transferência de Calor e Mecânica dos Fluidos Computacional, 2a Edição